In [52]:
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

start_date = dt.datetime(2000,1,1)
end_date = dt.datetime(2020,1,1)

GSPC = pd.DataFrame(pdr.get_data_yahoo('^GSPC', start_date, end_date, interval='m')['Adj Close']) #500 companies USA (S&P_500), USD
MXX = pd.DataFrame(pdr.get_data_yahoo('^MXX', start_date, end_date, interval='m')['Adj Close']) #Mexico exchange, currency in MXN
FCHI = pd.DataFrame(pdr.get_data_yahoo('^FCHI', start_date, end_date, interval='m')['Adj Close']) # France exchange,currency in EUR
NYA = pd.DataFrame(pdr.get_data_yahoo('^NYA', start_date, end_date, interval='m')['Adj Close']) #New york stock exhance, USD
JKSE = pd.DataFrame(pdr.get_data_yahoo('^JKSE', start_date, end_date, interval='m')['Adj Close'])#Jakarta exchange, currency in IDR
RUT = pd.DataFrame(pdr.get_data_yahoo('^RUT', start_date, end_date, interval='m')['Adj Close'])#2000 small companies, USD
KS11 = pd.DataFrame(pdr.get_data_yahoo('^KS11', start_date, end_date, interval='m')['Adj Close']) #Korea exchange, currency in KRW

In [53]:
#check length
len(GSPC), len(MXX), len(FCHI), len(NYA), len(JKSE), len(RUT), len(KS11)

(241, 241, 241, 241, 241, 241, 241)

In [54]:
# convert in USD, we will use fixed rates

for i in range(241):
    MXX.iloc[i]['Adj Close']  =  MXX.iloc[i]['Adj Close'] * 0.05
    FCHI.iloc[i]['Adj Close'] = FCHI.iloc[i]['Adj Close'] * 1.06
    JKSE.iloc[i]['Adj Close'] = JKSE.iloc[i]['Adj Close'] * 0.000064
    KS11.iloc[i]['Adj Close'] = KS11.iloc[i]['Adj Close'] * 0.000773

In [56]:
GSPC, MXX, KS11, FCHI

(              Adj Close
 Date                   
 2000-01-01  1394.459961
 2000-02-01  1366.420044
 2000-03-01  1498.579956
 2000-04-01  1452.430054
 2000-05-01  1420.599976
 ...                 ...
 2019-09-01  2976.739990
 2019-10-01  3037.560059
 2019-11-01  3140.979980
 2019-12-01  3230.780029
 2020-01-01  3225.520020
 
 [241 rows x 1 columns],
               Adj Close
 Date                   
 2000-01-01   329.283496
 2000-02-01   368.427490
 2000-03-01   373.662500
 2000-04-01   332.034009
 2000-05-01   298.057007
 ...                 ...
 2019-09-01  2150.563477
 2019-10-01  2166.864063
 2019-11-01  2141.008984
 2019-12-01  2177.050977
 2020-01-01  2205.415430
 
 [241 rows x 1 columns],
             Adj Close
 Date                 
 1999-12-31   0.729619
 2000-01-31   0.640338
 2000-02-29   0.665507
 2000-03-31   0.560726
 2000-04-30   0.565743
 ...               ...
 2019-08-31   1.594738
 2019-09-30   1.610530
 2019-10-31   1.613993
 2019-11-30   1.698799
 2019-12-31   1.6379